In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
class Node:
    pass


class NodeVisitor:

    def visit(self, node):
        method_name = f'visit_{type(node).__name__}'
        method = getattr(self, method_name, None)
        if method is None:
            method = self.generic_visit

        return method(node)

    def generic_visit(self, node):
        method_name = f'visit_{type(node).__name__}'
        raise RuntimeError(f"no method: '{method_name}'")


class Evaluator(NodeVisitor):

    def visit_Number(self, node):
        return node.value

    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)

    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)

    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)

    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)

    def visit_Neg(self, node):
        return -node.operand


class UnaryOp(Node):

    def __init__(self, operand):
        self.operand = operand


class BinOp(Node):

    def __init__(self, left, right):
        self.left = left
        self.right = right


class Add(BinOp):
    pass


class Sub(BinOp):
    pass


class Mul(BinOp):
    pass


class Div(BinOp):
    pass


class Neg(UnaryOp):
    pass


class Number(Node):
    def __init__(self, value):
        self.value = value


class StackCode(NodeVisitor):

    def generate_code(self, node):
        self.instructions = []
        self.visit(node)
        return self.instructions

    def visit_Number(self, node):
        self.instructions.append(('PUSH', node.value))

    def binop(self, node, instruction):
        self.visit(node.left)
        self.visit(node.right)
        self.instructions.append((instruction, ))

    def visit_Add(self, node):
        self.binop(node, 'ADD')

    def visit_Sub(self, node):
        self.binop(node, 'SUB')

    def visit_Mul(self, node):
        self.binop(node, 'MUL')

    def visit_Div(self, node):
        self.binop(node, 'DIV')

    def unaryop(self, node, instruction):
        self.visit(node.operand)
        self.instructions.append((instruction, ))

    def visit_Negate(self, node):
        self.unaryop(node, 'NEG')


"""

>>> t1 = Sub(Number(3), Number(4))
>>> t2 = Mul(Number(2), t1)
>>> t3 = Div(t2, Number(5))
>>> t4 = Add(Number(1), t3)
>>> e = Evaluator()
>>> e.visit(t4)
0.6

>>> s = StackCode()
>>> s.generate_code(t4)
[('PUSH', 1), ('PUSH', 2), ('PUSH', 3), ('PUSH', 4), ('SUB',), ('MUL',), ('PUSH', 5), ('DIV',), ('ADD',)]
"""  # noqa: E501

doctest.testmod()

TestResults(failed=0, attempted=8)